# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
#commented out ' + '/event_data'
filepath = os.getcwd() + '/event_data'
print(filepath)
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace
/home/workspace/event_data


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables
### The section below has been commented out - mentor pointed out in a message and gave us a dropbox link to download one
### large data file: https://www.dropbox.com/s/acrs9k66q34kwkb/event_datafile_new.csv?dl=0.
#### This section left on purpose, for continuity.

In [28]:
#Please refer to statement above - this section commented out
# initiating an empty list of rows that will be generated from each filefull_data_rows_list = [] 
    
# for every filepath in the file path list 
#for f in file_path_list:

# reading csv file 
#    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
#        # creating a csv reader object 
#        csvreader = csv.reader(csvfile) 
#        next(csvreader)
        
 # extracting each data row one by one and append it        
 #       for line in csvreader:
 #           #print(line)
 #           full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
#csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

#with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    #writer = csv.writer(f, dialect='myDialect')
    #writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                #'level','location','sessionId','song','userId'])
    #for row in full_data_rows_list:
        #if (row[0] == ''):
            #continue
        #writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [85]:
# check the number of rows in your csv file
print("number of rows in file:\n")
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

number of rows in file:

6821


 The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating Keyspace

In [5]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity
                        WITH REPLICATION = 
                        {'class': 'SimpleStrategy', 'replication_factor': 1}
                    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create 3 distinct queries to produce specified output

## QUERY 1:
### 1. Give the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
### For this query I used (session_id, item_in_session) as a composite partition key where session_id is the partition key and item_session is a clustering key, since the query asks for a specific session_id and item_in_session.

In [25]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS song_info (sessionid int, item_in_session int, artist_name text, song_length float, song_title text, PRIMARY KEY(sessionid, item_in_session))"
try:
    session.execute(query1)
except Exception as e:
    print(e)               

In [28]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info (sessionid, item_in_session, artist_name, song_length, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [45]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query_test1 = "SELECT sessionid, item_in_session, artist_name, song_length, song_title FROM udacity.song_info LIMIT 2"
try:
    rows = session.execute(query_test1)
except Exception as e:
    print(e)

print("test output to confirm that song_info table has data\n")
for row in rows:
    print(row.sessionid, row.item_in_session, row.artist_name, row.song_title, round(row.song_length,2))

test output to confirm that song_info table has data

23 0 Regina Spektor The Calculation (Album Version) 191.09
23 1 Octopus Project All Of The Champs That Ever Lived 250.96


In [44]:
# Actual query for song_info
query_song = "SELECT artist_name, song_title, song_length FROM song_info WHERE sessionid = 338 AND item_in_session = 4"
try:
    rows = session.execute(query_song)
except Exception as e:
    print(e)
  
print("This is the output for query to provide artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4\n")
print("{: <30} {: <40} {: <10}".format("ARTIST", "SONG", "DURATION\n"))

for row in rows:
    print("{: <30} {: <40} {: <10}".format(row.artist_name, row.song_title, round(row.song_length, 2)))

This is the output for query to provide artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

ARTIST                         SONG                                     DURATION
 
Faithless                      Music Matters (Mark Knight Dub)          495.31    


## QUERY 2:
### Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
### For this query I used composite Primary keys (userid, sessionid) to provide the best design for the query, since they will be how the result set will be stored.  The item_in_session field will be used for clustering and sorting, and in combination with the primary keys, provide the most efficient output for what is specified in the query.

In [63]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS artist_info (userid int, sessionid int,  item_in_session int, artist_name text, song_title text, \
          first_name text, last_name text, PRIMARY KEY((userid, sessionid), item_in_session))"
try:
    session.execute(query2)
except Exception as e:
    print(e)  

In [64]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_info (userid, sessionid, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))

In [71]:
query_test2 = "SELECT userid, sessionid, item_in_session, artist_name, song_title, first_name, last_name FROM udacity.artist_info LIMIT 2"
try:
    rows = session.execute(query_test2)
except Exception as e:
    print(e)

print("test output to confirm that artist_info table has data\n")
for row in rows:
    print(row.artist_name, row.song_title, row.first_name, row.last_name, row.userid, row.sessionid)

test output to confirm that artist_info table has data

System of a Down Sad Statue Emily Benson 58 768
Ghostland Observatory Stranger Lover Emily Benson 58 768


In [66]:
#Actual query for artist_info to get artist info for userid = 10 and sessionid = 182
query_artist = "SELECT userid, sessionid, item_in_session, artist_name, song_title, first_name, last_name  FROM artist_info WHERE userid = 10 AND sessionid = 182"
try:
    rows = session.execute(query_artist)
except Exception as e:
    print(e)
print("output of query to provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182\n")
print("{: <20} {: <55} {: <20} {: <20}".format("ARIST", "SONG", "USER FIRST NAME", "USER LAST NAME", "USER ID", "SESSION ID\n"))

for row in rows:
    print("{: <20} {: <55} {: <20} {: <20}".format(row.artist_name, row.song_title, row.first_name, row.last_name))

output of query to provide only the following: name of artist, song (sorted by itemInSession) and user (first and last name)## for userid = 10, sessionid = 182

ARIST                SONG                                                    USER FIRST NAME      USER LAST NAME      
Down To The Bone     Keep On Keepin' On                                      Sylvie               Cruz                
Three Drives         Greece 2000                                             Sylvie               Cruz                
Sebastien Tellier    Kilometer                                               Sylvie               Cruz                
Lonnie Gordon        Catch You Baby (Steve Pitron & Max Sanna Radio Edit)    Sylvie               Cruz                


## QUERY 3:
### 3. Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### For Query 3, the primary keys song_title and user_id (which provides unique users) provide the best means to get the combination of user and song information that the query asks for.

In [80]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS user_info (song_title text, userid int, item_in_session text, first_name text, last_name text, \
           PRIMARY KEY(song_title, userid))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [81]:
#create file for user info for users who listened to 'All Hands Against His Own'
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8')as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_info (song_title, userid, item_in_session, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[3], line[1], line[4]))


In [82]:
#test to see if any data was inserted into user_info table
query_test3 = "SELECT * FROM udacity.user_info LIMIT 2"
try:
    count3 = session.execute(query_test3)
except Exception as e:
    print(e)

print("test output to confirm that user_info table has data\n")
for row in count3:
    print(row.song_title, row.userid, row.item_in_session, row.first_name, row.last_name)

test output to confirm that user_info table has data

Wonder What's Next 49 3 Chloe Cuevas
In The Dragon's Den 49 41 Chloe Cuevas


In [83]:
query_user = "SELECT userid, first_name, last_name, song_title FROM user_info WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query_user)
except Exception as e:
    print(e)

print("{: <20} {: <20} {: <50}".format("USER FIRST NAME", "USER LAST NAME", "QUERY SONG\n"))

for row in rows:
    print("{: <20} {: <20} {: <20}".format(row.first_name, row.last_name, row.song_title))

USER FIRST NAME      USER LAST NAME       QUERY SONG
                                       
Jacqueline           Lynch                All Hands Against His Own
Tegan                Levine               All Hands Against His Own
Sara                 Johnson              All Hands Against His Own


### Drop the tables before closing out the sessions

In [24]:
## TO-DO: Drop the table song_info before closing out the sessions
drop_query1 = "DROP TABLE IF EXISTS song_info"
try:
    session.execute(drop_query1)
except Exeption in e:
    print(e)

In [62]:
## TO-DO: Drop the table artist_info before closing out the sessions
drop_query1 = "DROP TABLE IF EXISTS artist_info"
try:
    session.execute(drop_query1)
except Exeption in e:
    print(e)

In [76]:
## TO-DO: Drop the table user_info before closing out the sessions
drop_query3 = "DROP TABLE IF EXISTS user_info"
try:
    session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()